In [1]:
import os

In [4]:
%pwd

'c:\\Users\\saadp\\Desktop\\Inquizty_WorkDone\\Others\\Text-Summarizer-Project'

In [3]:
os.chdir("../")

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [11]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [12]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([Path(self.config.artifacts_root)])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([Path(config.root_dir)])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir = Path(config.root_dir),
            source_URL = config.source_URL,
            local_data_file = Path(config.local_data_file),
            unzip_dir = Path(config.unzip_dir)
        )
        
        return data_ingestion_config

In [13]:
import os
import urllib.request as request
import zipfile
from textSummarizer.utils.common import get_size
from textSummarizer.logging import logger

In [14]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
    def download_data(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} downloaded! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
            
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok = True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        logger.info(f"File extracted at : {unzip_path}")


In [23]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_data()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-09-06 13:04:27,779 - INFO - common - YAML file config\config.yaml loaded successfully.]
[2025-09-06 13:04:27,784 - INFO - common - YAML file params.yaml loaded successfully.]
[2025-09-06 13:04:27,786 - INFO - common - Created directory at: artifacts]
[2025-09-06 13:04:27,787 - INFO - common - Created directory at: artifacts\data_ingestion]


[2025-09-06 13:04:32,051 - INFO - 1366787839 - artifacts\data_ingestion\data.zip downloaded! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 8B74:08E8:60B2F:10A97F:68BBE404
Accept-Ranges: bytes
Date: Sat, 06 Sep 2025 07:34:29 GMT
Via: 1.1 varnish
X-Served-By: cache-bom-vanm7210029-BOM
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1757144069.926703,VS0,VE922
Vary: Authorization,Accept-Encoding
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: b82cdd54f7ac8b29733e7e10104f575ec62cf5ca
Expires: Sat, 06 Sep 2025 07:39:29 GMT
Source-Age: 1

]
[2025-09-06 13:04:32,216 - INFO - 1366787